Importing Dataset
Before MLDatasets, make sure it is downloaded via Julia ], add MLDatasets

In [ ]:
using MLDatasets

# https://juliaml.github.io/MLDatasets.jl/stable/datasets/CIFAR10/#MLDatasets.CIFAR10.traintensor

# load full training set as float 32, otherwise get warnings in train
train_x, train_y = CIFAR10.traindata(Float32) # full traindatatset 50000 training images + 10 classes
# boobs
# load full test set
test_x, test_y = CIFAR10.testdata(Float32) # full test dataset 10000 test images + 10 classes

Import libraries 

In [ ]:
using Flux
using Flux: Data.DataLoader
using Flux: @epochs, onehotbatch, crossentropy, Momentum, update!, onecold, ADAM
using Plots
using CUDA
using Statistics

Plot an Image with a Title and vlaues

In [ ]:
CIFAR10.trainlabels(1)

In [ ]:
CIFAR10.classnames()

In [ ]:
CIFAR10.classnames()[CIFAR10.trainlabels(1) + 1]

In [ ]:
plot(CIFAR10.convert2image(CIFAR10.traintensor(1)),  title = CIFAR10.classnames()[CIFAR10.trainlabels(1) + 1])

Image info with Numerical Values

In [ ]:
CIFAR10.traintensor(Float32, 1:50) # original values are gray scale 32 x 32 image size with RGB (3)

Train 50 Dataset (Demonstration)
Conv(filter, in => out, σ = identity; stride = 1, pad = 0, dilation = 1, groups = 1, [bias, weight, init])

First Group

In [5]:
layer1 = Flux.Conv((3,3), 3=>32, relu, pad=SamePad()) # num of paras = 3*(3*3)*32 = 864
layer2 = Flux.BatchNorm(32) # this does not change when a batch is 1
layer3 = Flux.Conv((3,3), 32=>32, relu, pad=SamePad())
layer4 = Flux.BatchNorm(32)
layer5 = Flux.MaxPool((2, 2))

MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2))

Second Group

In [6]:
layer6 = Flux.Conv((3,3), 32=>64, relu, pad=SamePad()) # num of paras = 3*(3*3)*32 = 864
layer7 = Flux.BatchNorm(64) # this does not change when a batch is 1
layer8 = Flux.Conv((3,3), 64=>64, relu, pad=SamePad())
layer9 = Flux.BatchNorm(64)
layer10 = Flux.MaxPool((2, 2))

MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2))

Full Connection

In [7]:
# before this Flatten is called

layer11 = Flux.Dropout(0.2)
layer12 = Flux.Dense(4096, 256, relu) # (8*8)*64 features
layer13 = Flux.Dense(256, 10)

# layer13 goes to softmax

Dense(256, 10)

In [ ]:
r1 = layer1(CIFAR10.traintensor(Float32, 1:50))

In [ ]:
r2 = layer2(r1)

In [ ]:
r3 = layer3(r2)

In [ ]:
r4 = layer4(r3)

In [ ]:
r5 = layer5(r4)

In [ ]:
r6 = layer6(r5)

In [ ]:
r7 = layer7(r6)

In [ ]:
r8 = layer8(r7)

In [ ]:
r9 = layer9(r8)

In [ ]:
r10 = layer10(r9)

In [ ]:
full_1 = Flux.flatten(r10)

In [ ]:
full_2 = layer11(full_1)

In [ ]:
full_3 = layer12(full_2)

In [ ]:
result = layer13(full_3)

In [ ]:
softmax(result) # check the 6th has highest value (those values are probabilities)

Creating CNN: Select 1 Model

Simple Model

In [ ]:
model = Flux.Chain(
    # input size = 32*32*3*1 

    Flux.Conv((3,3), 3=>32, relu, pad=SamePad()),
    Flux.BatchNorm(32), # should be the size of the channel dimension in your data
    Flux.Conv((3,3), 32=>32, relu, pad=SamePad()),
    Flux.BatchNorm(32),
    Flux.MaxPool((2, 2)), # 16*16*32*1

    Flux.flatten, 
    Flux.Dropout(0.2),

    Flux.Dense(8192, 64, relu), # (16*16)*32 features
    Flux.Dense(64, 10),
    softmax
    ) |> gpu

Full Model

In [ ]:
model = Flux.Chain(
    # input size = 32*32*3*1 

    Flux.Conv((3,3), 3=>32, relu, pad=SamePad()),
    Flux.BatchNorm(32), # should be the size of the channel dimension in your data
    Flux.Conv((3,3), 32=>32, relu, pad=SamePad()),
    Flux.BatchNorm(32),
    Flux.MaxPool((2, 2)), # 16*16*32*1

    Flux.Conv((3,3), 32=>64, relu, pad=SamePad()), # 16*16*64*1
    Flux.BatchNorm(64), # should be the size of the channel dimension in your data
    Flux.Conv((3,3), 64=>64, relu, pad=SamePad()),
    Flux.BatchNorm(64),
    Flux.MaxPool((2, 2)), # 8*8*64

    Flux.Conv((3,3), 64=>128, relu, pad=SamePad()), # 8*8*128*1
    Flux.BatchNorm(128), # should be the size of the channel dimension in your data
    Flux.Conv((3,3), 128=>128, relu, pad=SamePad()),
    Flux.BatchNorm(128),
    Flux.MaxPool((2, 2)), # 4*4*128*1

    Flux.flatten, 
    Flux.Dropout(0.2),

    Flux.Dense(2048, 516, relu), # (4*4)*128 features
    Flux.Dense(516, 10),
    softmax
    ) |> gpu

VGG16 Model : Consuming a lot of Memory (RAM)

In [ ]:
model = Chain
(
  Conv((3, 3), 3 => 64, relu, pad=(1, 1), stride=(1, 1)), 
  BatchNorm(64),
  Conv((3, 3), 64 => 64, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(64),
  MaxPool((2,2)),

  Conv((3, 3), 64 => 128, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(128),
  Conv((3, 3), 128 => 128, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(128),
  MaxPool((2,2)),

  Conv((3, 3), 128 => 256, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(256),
  Conv((3, 3), 256 => 256, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(256),
  Conv((3, 3), 256 => 256, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(256),
  MaxPool((2,2)),

  Conv((3, 3), 256 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  MaxPool((2,2)),

  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  MaxPool((2,2)),

  x -> reshape(x, :, size(x, 4)),
  Dense(512, 4096, relu),  
  Dropout(0.5),

  Dense(4096, 4096, relu),
  Dropout(0.5),
  
  Dense(4096, 10),
  softmax
  ) |> gpu

ResNet Model: Must Run All Cells 

In [ ]:
struct conv_skip
    conv::Chain
end |> gpu
conv_skip(input_filter, filter) = conv_skip(shortcut(input_filter, filter))

In [4]:
function (op::conv_skip)(x, y)
    z = op.conv(y)
    return x + z
end

In [5]:
identityBlock(input_filter, filter) = Chain(
    # layer 1
    Conv((3, 3), input_filter => filter, pad=(1, 1), stride=(1, 1)), # pad=(1, 1) is same
    BatchNorm(filter, relu),

    # layer 2
    Conv((3, 3), filter => filter, pad=(1, 1), stride=(1, 1)), # pad=(1, 1) is same
    BatchNorm(filter, relu),
    
    # layer 3
    Conv((3, 3), filter => filter, pad=(1, 1), stride=(1, 1)), # pad=(1, 1) is same
    BatchNorm(filter),    
) |> gpu

identityBlock (generic function with 1 method)

In [ ]:
shortcut(input_filter, filter) = Chain(
  Conv((3,3), input_filter => filter, stride=(1,1)), # pad=0 is valid
  BatchNorm(filter),
) |> gpu

In [ ]:
convBlock(input_filter, filter) = Chain(
    # layer 1
    Conv((3, 3), input_filter => filter, pad=(1, 1), stride=(1, 1)), # pad=(1, 1) is same
    BatchNorm(filter, relu),

    # layer 2
    Conv((3, 3), filter => filter, pad=(1, 1), stride=(1, 1)), # pad=(1, 1) is same
    BatchNorm(filter, relu),
    
    # layer 3
    Conv((3, 3), filter => filter, stride=(1, 1)), # pad=0 is valid
    BatchNorm(filter),
) |> gpu

In [ ]:
model = Chain(
    # input size = 32*32*3*1 
    
    # prepare for ResNet
    Conv((7,7), 3=>64, stride=(2,2)), # output = 13*13*64 : params = 3*7*7*64: 
    BatchNorm(64, relu),
    MaxPool((3,3), pad=(1,1), stride=(2,2)), # output = 7*7*64
    
    # ResNet 18 layer: The simplest "ResNet"-type connection is just SkipConnection(layer, +)
    SkipConnection(convBlock(64, 64), conv_skip(64, 64)), # output = 5*5*64

    # *identityBlock must have the same input_filter and filter size. Otherwiese it gives error identityBlock(64, 128) -> error
    SkipConnection(identityBlock(64, 64), +), # output = 5*5*64
    SkipConnection(identityBlock(64, 64), +), # output = 5*5*64
     
    SkipConnection(convBlock(64, 128), conv_skip(64, 128)), # output = 3*3*128
    SkipConnection(identityBlock(128, 128), +), # output = 3*3*128
    SkipConnection(identityBlock(128, 128), +), # output = 3*3*128

    # full connections layers
    MaxPool((2, 2)), # 1*1*128
    Flux.flatten, 
    Dropout(0.3),    
    Dense(128, 1024, relu), 
    Dropout(0.3),
    Dense(1024, 10),
    
    softmax
) |> gpu


Training Data

Data Preparation ------------------------------

In [ ]:
# without CUDA (CPU)

# MLDatasets returns UInt8 thus convert it to Float64
train_x = Array{Float32}(train_x) # without converting, it gives errors, especially, the labels
test_x = Array{Float32}(test_x)

# construct one-hot vectors from labels
train_y = onehotbatch(train_y, 0:9) # https://fluxml.ai/Flux.jl/stable/data/onehot/
test_y = onehotbatch(test_y, 0:9) # This is a sparse matrix, which stores just a Vector{UInt32} containing the indices of the nonzero elements.

train_data = DataLoader(train_x, train_y, batchsize=50, shuffle=true) # 1000 blocks
test_data = DataLoader(test_x, test_y, batchsize=10) # 100 blocks


# CUDA tensor (GPU)

# train_x_tensor = permutedims(train_x, [1, 2, 3, 4])
# train_y_onehot = onehotbatch(train_y, 0:9)

# test_x_tensor = permutedims(test_x, [1, 2, 3, 4])
# test_y_onehot = onehotbatch(test_y, 0:9)

# cu_train_x_tensor = cu(train_x_tensor)
# cu_train_y_onehot = cu(train_y_onehot)

# cu_test_x_tensor = cu(test_x_tensor)
# cu_test_y_onehot = cu(test_y_onehot)

# train_data = DataLoader(cu_train_x_tensor, cu_train_y_onehot, batchsize=50, shuffle=true) # 1000 blocks
# test_data = DataLoader(cu_test_x_tensor, cu_test_y_onehot, batchsize=10) # 100 blocks

println("conversion is done")

Optimizer ------------------------------

In [ ]:
lr = 0.001 # learning_rate
opt = ADAM(lr, (0.9, 0.999)) |> gpu

Loss and Accuracy ------------------------------

In [ ]:
# loss is for softmax
loss(x, y) = sum(crossentropy(model(x), y))  # when this loss is called, assume x and y are given
accuracy(x, y) = mean(onecold(model(x), 0:9) .== onecold(y, 0:9))  # when this accuracy is called, assume x and y are given

# for plotting
train_losses = []
test_losses = []
train_acces = []
test_acces = []

Callbacks ------------------------------

In [ ]:
function loss_all(data_loader)
    sum([loss(x, y) for (x,y) in data_loader]) / length(data_loader) 
    # using sum twice: 
    # first sum in loss(x, y) is for batched values (each batch 50 images)
    # second sum 2 lines aboive is for all batches (1000 batches)
end

function acc(data_loader)         
    sum([accuracy(x, y) for (x,y) in data_loader]) / length(data_loader) 
end

Training ------------------------------

In [ ]:
epochs = 2 # CPU takes a lot of time and memory for each epoch
# epochs = 50 # GPU must be on

for epoch = 1:epochs

    @info "epoch" epoch
	
	num = 0
	
	for (x, y) in train_data

		batch = (x, y)
		gs = gradient(params(model)) do
			l = loss(batch...)
		end

		num += 1
		fin = (num/length(train_data))*100

		print("...Training...")
		print(round(fin; digits=3))
		print("%\n")

		Flux.update!(opt, params(model), gs)
		
	end

	@info "...Calculating..."
	push!(train_losses, loss_all(train_data)),
    push!(test_losses, loss_all(test_data)),
    push!(train_acces, acc(train_data)),
    push!(test_acces, acc(test_data))  

	@show train_loss = loss_all(train_data)
	@show test_loss = loss_all(test_data)
	@show train_acc = acc(train_data)
	@show test_acc = acc(test_data)
end

Plotting ------------------------------

In [ ]:
plot([train_losses, test_losses], title = "Loss", label = ["Training" "Test"])

In [ ]:
plot([train_acces, test_acces], title = "Accuracy", label = ["Training" "Test"])